<a href="https://colab.research.google.com/github/gkunal8019/Assignment/blob/main/amazon_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [26]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"
    #URL= "https://www.amazon.in/s?rh=n%3A1375424031&fs=true&ref=lp_1375424031_sar"
    #URL = "https://www.amazon.com/s?bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&dc&fst=as%3Aoff&pd_rd_r=5c04573e-5d2a-4f40-ac0b-fd2a5c6f501a&pd_rd_w=mU6qz&pd_rd_wg=5D6fv&pf_rd_p=5b7fc375-ab40-4cc0-8c62-01d4de8b648d&pf_rd_r=6616WNB48T67V25MMP1E&qid=1602294815&rnid=16225007011&ref=pd_gw_unk"
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [27]:
amazon_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         42 non-null     object
 1   price         42 non-null     object
 2   rating        42 non-null     object
 3   reviews       42 non-null     object
 4   availability  42 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB


In [28]:
amazon_df.to_csv('amazon_df.txt', sep='\t', index=False)

In [29]:
with open('/content/amazon_df.txt') as f:
    lines = f.readlines()

In [30]:
lines

['title\tprice\trating\treviews\tavailability\n',
 'Sony Playstation PS4 1TB Black Console\t\t4.7 out of 5 stars\t968 ratings\tIn Stock\n',
 'PlayStation 4 500GB Console (Renewed)\t\t4.0 out of 5 stars\t381 ratings\tOnly 5 left in stock - order soon.\n',
 'PlayStation 4 Slim 1TB Console (Renewed)\t\t4.3 out of 5 stars\t1,019 ratings\tOnly 20 left in stock - order soon.\n',
 'PlayStation 4 Slim 500GB Console [Discontinued] (Renewed)\t\t4.5 out of 5 stars\t1,243 ratings\tIn Stock.\n',
 'PlayStation 4 Slim 1TB Console - Black (Renewed)\t\t4.5 out of 5 stars\t1,211 ratings\tIn Stock.\n',
 'Sony PlayStation 2 Console - Black (Renewed)\t\t4.3 out of 5 stars\t1,099 ratings\tOnly 14 left in stock - order soon.\n',
 'PlayStation 4 Slim 1TB Console (Renewed)\t\t4.3 out of 5 stars\t1,019 ratings\tOnly 20 left in stock - order soon.\n',
 'PlayStation 2 Slim Console PS2 (Renewed)\t\t4.2 out of 5 stars\t936 ratings\tOnly 3 left in stock - order soon.\n',
 'Playstation SONY 4, 500GB Slim System [CUH-